Figuring out how to output roundness/sharpness, and looking into
what happens when using two different aperture sizes.

In [1]:
import numpy as np
from matplotlib import pyplot as plt

from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils import aperture_photometry,CircularAnnulus,CircularAperture
from photutils import DAOStarFinder

In [3]:
# image = 'hor1DRCs/jdan20010_drc.fits'
# hdu = fits.open(image)
# sci = hdu[1].data
# hdr = hdu[0].header
# tab = hdu[4].data
# photflam = hdu[1].header['PHOTFLAM']

# data = hdu[1].data
# hdu.close()

# temp_sci=data*hdr['texptime']

# fits.writeto('hor1DRCs/F814W_ctsALL_raw.fits', temp_sci, header=hdr)

In [4]:
image = 'hor1DRCs/F814W_ctsALL_raw.fits'
# fits.info(image)

hdu = fits.open(image)
sci = hdu[0].data
hdr = hdu[0].header
hdu.close()

In [5]:
data = sci.copy()

In [6]:
mean, median, std = sigma_clipped_stats(data, sigma=3.0, maxiters=5)
print(mean, median, std)

0.49618652 -0.06826966 19.541706


In [7]:
daofind = DAOStarFinder(fwhm=2., threshold=3.*std)
sources = daofind(data - median)
print(len(sources))

43394


In [8]:
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures_r3 = CircularAperture(positions, r=3.)

rawflux_r3 = aperture_photometry(data, apertures_r3)

In [14]:
rawflux_r3['roundness1']= sources['roundness1']
rawflux_r3['roundness2']= sources['roundness2']
rawflux_r3['sharpness']= sources['sharpness']

In [15]:
annulus_apertures = CircularAnnulus(positions, r_in=9., r_out=12.)

annulus_masks = annulus_apertures.to_mask(method='center')

bkg_median = []
for mask in annulus_masks:
    
    annulus_data = mask.multiply(data)
    annulus_data_1d = annulus_data[mask.data > 0]
    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
    bkg_median.append(median_sigclip)

bkg_median = np.array(bkg_median)

In [16]:
rawflux_r3['annulus_median'] = bkg_median
rawflux_r3['aper_bkg'] = bkg_median*apertures_r3.area
rawflux_r3['final_phot'] = rawflux_r3['aperture_sum'] - rawflux_r3['aper_bkg']